# Quadratic Programming

Quadratic Programs (QPs) have objective functions with squared terms.  The canonical formulation of a QP is:

$\min{\frac{1}{2} x^T Q x + c^T x}$

$\begin{eqnarray*}
\text{s.t.} \quad Ax &\leq& b \\
Cx &=& d 
\end{eqnarray*}$

where $x$ is a column vector.

In [ ]:
import numpy as np

data = np.genfromtxt('ozone.csv', delimiter=',', skip_header=1)
data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

heading = ['ozone', 'radiation', 'temperature', 'wind']

fig,ax = plt.subplots()
ax.scatter(data[:,3], data[:,0], c='k', alpha=0.75)
ax.set_xlabel(heading[1])
ax.set_ylabel(heading[0])
fig.suptitle('Ozone versus Wind')
plt.show()

# Regression in Python

Regression can be performed with several packages in Python, among which are <code>sklearn</code> and <code>scipy.stats</code>.  I use the latter because I find its use to be more intuitive.  If you choose the latter it can be imported with thsi statement:

``` python
from sklearn.linear_model import LinearRegression
```

In [ ]:
from scipy.stats import linregress

result = linregress(x=data[:,3], y=data[:,0])
print(result)

# Regression using Quadratic Programming  with Gurobi

In [ ]:
import gurobipy as grb
import numpy as np

n = data.shape[0]

# Create Gurobi model
model = grb.Model('QP-Regression')

''' Set up data '''
x = data[:,3]
y = data[:,0]

# Create matrix of decision variables
''' Note that by default a continous variable's lower bound is 0.0.
    Where decision variables can be negative, the lower bound needs to be reset. '''
m = model.addVar(lb= -float('inf'), vtype=grb.GRB.CONTINUOUS, name='slope') 
b = model.addVar(lb= -float('inf'), vtype=grb.GRB.CONTINUOUS, name='intercept') 
model.update()

# Create constraints
''' This problem has no constraints '''

# Create objective function
model.setObjective((m*x+b)@np.identity(x.shape[0])@(m*x+b) - 2*y@(m*x+b), grb.GRB.MINIMIZE)  # no need to include the constants term y@y
model.update()

# Optimize the model
model.optimize()

''' Print decision variable values and other information '''
for var in model.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}')

## Alternate formulation

In [ ]:
import gurobipy as grb
import numpy as np

n = data.shape[0]

# Create Gurobi model
model = grb.Model('QP-Regression')

# Create matrix of decision variables
''' Note that by default a continous variable's lower bound is 0.0.
    Where decision variables can be negative, the lower bound needs to be reset. '''
m = model.addVar(lb= -float('inf'), vtype=grb.GRB.CONTINUOUS, name='slope') 
b = model.addVar(lb= -float('inf'), vtype=grb.GRB.CONTINUOUS, name='intercept') 

# Set up difference vector
d = m*data[:,3] + b - data[:,0]
model.update()

# Create constraints
''' This problem has no constraints '''

# Create objective function
model.setObjective(d @ d, grb.GRB.MINIMIZE)
model.update()

# Optimize the model
model.optimize()

''' Print decision variable values and other information '''
for var in model.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}')

Note that the slope and intercept from the two methods are identical.

# Optimizing Investment Portfolios

In [ ]:
import gurobipy as gpy
import matplotlib.pyplot as plt
import numpy as np

Q = np.genfromtxt('Q.txt')
r = np.genfromtxt('r.txt')

num_stock = r.shape[0]
B = 100.0
max_r = r.max()
num_level = 10
r_inc = (r.max() - (r.max()+r.min())/2)/num_level
results = []

''' Two models can be used to determine the efficient frontier
    of invesment portfolios.  The first minimizes risk for a
    specified level of return.  The second formulation exchanges
    roles between the objective function and the constraints by
    maximizing return for a given level of risk. 
    
    The latter model is shown in the lines that are commented out. 
    The loop that generates the efficient frontier works only for 
    the former formulation and would need to be revised for the latter. '''

''' Define Gurobi model '''
m = gpy.Model("QP-Proto")

''' Define investment decision variables '''
x = m.addMVar((num_stock,), vtype=gpy.GRB.CONTINUOUS)
m.update()

''' Define objective function '''
m.setObjective(x@Q@x, gpy.GRB.MINIMIZE)
#m.setObjective(gpy.quicksum(r @ x / B for i in range(num_stock)),gpy.GRB.MAXIMIZE)
m.update()

''' Add constraints '''
m.addConstr(x.sum() == B, name='Portfolio Budget')
m.addConstr((x@r)/B >= max_r, name="Return target")
#m.addConstr(x@Q@x,gpy.GRB.LESS_EQUAL, max_risk, name='Quadratic Risk Constraint')

m.optimize()

if m.status == 2:
    results.append([max_r,m.ObjVal])
    print('Results for Initial Model')
    print('\n'.join([f'{v.varName}, {v.x}' for v in x]))
else:
    print('Initial model infeasible')
    
for i in range(num_level):
    m.getConstrByName('Return target').RHS -= r_inc # decrease return target
    m.update()
    m.optimize()
    if m.status == 2:
        results.append([m.getConstrByName('Return target').RHS,m.ObjVal])
    else:
        print( "Model ",str(i)," Infeasible")
        
''' Annulaize return and risk, normalize risk also'''
for i in range(len(results)):
    results[i][1] = (12*results[i][1])**0.5/B
    results[i][0] = (1.0 + results[i][0]) ** 12.0 - 1.0

''' Plot results '''
fig,ax = plt.subplots()
ax.plot(*zip(*results), c='k')
ax.scatter(*zip(*results), c='k')
fig.set_size_inches(10,7)
ax.set_ylabel('Expected Annual Return')
ax.set_xlabel('Standard Deviation of Return Per Dollar')
ax.set_title('Portfolio Optimization',fontsize = 22)
ax.xaxis.label.set_fontsize(20)
ax.yaxis.label.set_fontsize(20)
ax.set_xlim(min([x[0] for x in results])*0.9,max([x[0] for x in results])*1.1)
ax.set_ylim(min([x[1] for x in results])*0.9,max([x[1] for x in results])*1.1)
ax.tick_params(axis='both',labelsize =14)
ax.tick_params(axis='both',which='both',top=False,right=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('ef.jpg', dpi=300)
plt.show()